In [1]:
# Import libraries

import numpy as np
import matplotlib.pyplot as plt

from data import pipeline
from data.window import SlidingWindow

# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorflow as tf
import datetime

# Clear any logs from previous runs
# rm -rf ./logs/

# Data preparation

In [2]:
X_train, Y_train, X_test, Y_test = pipeline.load("LSTM") # load / buildAndSave / build
print(X_train.shape, Y_train.shape)

Loading LSTM dataset
Done!
(1050578, 4, 8) (1050578, 1)


In [3]:
from sklearn.preprocessing import StandardScaler

X_train = X_train.reshape(-1, 32)
X_test = X_test.reshape(-1, 32)

sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

sc_y = StandardScaler()
Y_train = sc_y.fit_transform(Y_train.reshape(-1, 1))

X_train = X_train.reshape(-1, 4, 8)
X_test = X_test.reshape(-1, 4, 8)

In [4]:
print(X_train.shape, Y_train.shape)

(1050578, 4, 8) (1050578, 1)


# Training

In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.losses import MeanSquaredError
from keras.optimizers import Adam
import keras_tuner as kt

In [6]:
def build_model(hp):
    model = Sequential()
    
    # Tune the number of units in the LSTM per cell in the layer
    # hp_units between 50-90
    hp_units = hp.Int('units', min_value= 50, max_value= 90, step= 2)
    model.add(LSTM(units= hp_units, activation= 'relu', input_shape= (4, 8)))
    model.add(Dense(1))

    # Tune the learning rate for the optimizer
    # hp_learning_rate from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(optimizer= Adam(learning_rate= hp_learning_rate),
                  loss= ['mean_squared_error'],
                  metrics= [MeanSquaredError()])
    return model

tuner = kt.BayesianOptimization(
    hypermodel= build_model,
    objective= kt.Objective("val_mean_squared_error", direction= "min"),
    num_initial_points= 2,
    executions_per_trial= 2,
    project_name= "lstm_automl",
    overwrite= True # True when searching
)

In [ ]:
# Reload tuner
# tuner.reload()

tuner.search(x= X_train, y= Y_train, epochs= 10)

tuner.results_summary()

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""The best:
- dimension of the hidden state found is {best_hps.get('units')}
- learning rate found is {best_hps.get('learning_rate')}""")


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
74                |?                 |units
0.00017441        |?                 |learning_rate

Epoch 1/10
32831/32831 [==============================] - 122s 4ms/step - loss: 0.1159 - mean_squared_error: 0.1159
Epoch 2/10
32831/32831 [==============================] - 120s 4ms/step - loss: 0.0873 - mean_squared_error: 0.0873
Epoch 3/10
32831/32831 [==============================] - 117s 4ms/step - loss: 0.0797 - mean_squared_error: 0.0797
Epoch 4/10
32831/32831 [==============================] - 121s 4ms/step - loss: 0.0745 - mean_squared_error: 0.0745
Epoch 5/10
32831/32831 [==============================] - 151s 5ms/step - loss: 0.0707 - mean_squared_error: 0.0707
Epoch 6/10
32831/32831 [==============================] - 124s 4ms/step - loss: 0.0676 - mean_squared_error: 0.0676
Epoch 7/10
32831/32831 [==============================] - 125s 4ms/step - loss: 0.0651 - mean_squared_error: 0.0651
Epoch 8/10


In [ ]:
# Reload tuner
tuner.reload()

model = tuner.hypermodel.build(best_hps)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(X_train, Y_train, epochs= 10, callbacks= [tensorboard_callback])

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

%tensorboard --logdir logs/fit

print('Loss (RMSE) : %.3f' % sqrt(mean_squared_error(Y_train, model.predict(X_train))))

# Evaluation

In [ ]:
X_test = X_test[:1000]
Y_test = Y_test[:1000]
print(X_test.shape, Y_test.shape)

In [ ]:
sw = SlidingWindow(X_test, reset_cycle= 37)

pred = model.predict(sw.values.reshape(-1, 4, 8), verbose=0) # iterator: 0
Y_preds = [pred]

last_progression = 0
print(0, '%')
while sw.next(pred): # iterator: t-1
    arg = sw.values.reshape(-1, 4, 8)
    pred = model.predict(sw.values.reshape(-1, 4, 8), verbose=0) # iterator: t
    if np.isnan(pred):
        print(arg)
    Y_preds.append(pred)
        
    progression = (sw.it+1) / len(X_test) * 100
    if progression - last_progression > 1 and not int(progression) % 10:
        print(int(progression), '%')
        last_progression = progression

Y_preds = np.array(Y_preds).reshape(-1)

In [ ]:
print('Test RMSE : %.3f' % sqrt(mean_squared_error(Y_test, Y_preds)))

In [ ]:
Y_preds = sc_y.inverse_transform(Y_preds.reshape(1, -1))
Y_preds = Y_preds.reshape(-1)

year = 5
plt.plot([decade for decade in range(37)], Y_test[37*year:37*(year+1)], color="blue", label= "raw")
plt.plot([decade for decade in range(37)], Y_preds[37*year:37*(year+1)], color="red", label= "lstm")
plt.plot([decade for decade in range(37)], Y_naive[37*year:37*(year+1)], color="red", label= "naive model")
plt.xlabel("Decades")
plt.ylabel("Growth")
plt.legend()
plt.show()